In [15]:
import librosa
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical




In [16]:
DATA_PATH = "./data/"

# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [54]:
print()

print( to_categorical(np.arange(0, len(os.listdir(DATA_PATH)))))


[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [18]:
# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [ ]:
wave, sr = librosa.load()

In [29]:

def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_len=max_pad_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

In [65]:
path=DATA_PATH

max_pad_len=11

labels, _, _ = get_labels(path)

label = labels[0]

wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
print(wavfiles[1])


./data/cat/6f689791_nohash_0.wav


In [34]:
def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state)

In [68]:
labels, indices, _ = get_labels(DATA_PATH)

print(indices)

[0 1 2]


In [23]:
def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data

In [24]:
def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]


In [69]:
data = prepare_dataset(path)


In [70]:
type(data)

dict

In [31]:
# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_pad_len=feature_dim_2)

In [47]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [35]:
# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

/home/jmnt/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(1713, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/jmnt/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(1742, 2) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [40]:
# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3

In [41]:
# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

In [42]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [43]:
print(y_train_hot)

[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 ..., 
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


In [44]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [45]:
# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [48]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/50
3112/3112 [==============================] - 6s - loss: 1.6734 - acc: 0.4026 - val_loss: 0.9867 - val_acc: 0.4494
Epoch 2/50
3112/3112 [==============================] - 4s - loss: 0.9539 - acc: 0.5504 - val_loss: 0.7903 - val_acc: 0.6180
Epoch 3/50
3112/3112 [==============================] - 4s - loss: 0.7691 - acc: 0.6726 - val_loss: 0.5492 - val_acc: 0.8198
Epoch 4/50
3112/3112 [==============================] - 5s - loss: 0.6146 - acc: 0.7548 - val_loss: 0.6156 - val_acc: 0.7620
Epoch 5/50
3112/3112 [==============================] - 5s - loss: 0.5616 - acc: 0.7760 - val_loss: 0.4355 - val_acc: 0.8348
Epoch 6/50
3112/3112 [==============================] - 4s - loss: 0.4891 - acc: 0.8123 - val_loss: 0.4677 - val_acc: 0.8112
Epoch 7/50
3112/3112 [==============================] - 4s - loss: 0.4352 - acc: 0.8400 - val_loss: 0.3549 - val_acc: 0.8704
Epoch 8/50
3112/3112 [==============================] - 4s - loss: 0.4076 - a

In [49]:
print(predict('./data/bed/004ae714_nohash_1.wav', model=model))

bed
